In [14]:
#讀取Excel
from win32com.client import Dispatch
def findCustCD(x):
    import pandas as pd
    from win32com.client import Dispatch
    
    '''測試用log 非def時測試
    #print('請輸入:')
    #x=float(input())
    #print(type(x))
    #print('x=',x,'迴圈開始\n\n')
    '''

    xlsApp = Dispatch("Excel.Application")
    xlsApp.Visible = 0
    xlsBook = xlsApp.WorkBooks.Open(r'C:\Excel.xlsx', UpdateLinks=False)

    xlslist=[]
    xlsCurSheet = xlsBook.WorkSheets("sheet1")
    for i in range(1, xlsCurSheet.UsedRange.Rows.Count + 1):
        if xlsCurSheet.Cells(i,1).Value == float(x):
            #檢查標籤型別有沒有資料
            lbtype=[11,14,17]
            for j in lbtype:
                if type(xlsCurSheet.Cells(i,j).Value) !=str:
                    xlsCurSheet.Cells(i,j).Value = 'N/A'
                
                    '''測試用log
                    else:
                    print('儲存格#N/A檢查=',type(xlsCurSheet.Cells(i,j).Value))
                    '''
                    
            value= [
                str(int(xlsCurSheet.Cells(i,1).Value)),#暫時改回str,因為要寫成html,要跟excel比對的時候再用int
                '00000'+str(int(xlsCurSheet.Cells(i,4).Value)),
                xlsCurSheet.Cells(i,11).Value,
                xlsCurSheet.Cells(i,12).Value,
                xlsCurSheet.Cells(i,13).Value,
                xlsCurSheet.Cells(i,14).Value,
                xlsCurSheet.Cells(i,15).Value,
                xlsCurSheet.Cells(i,16).Value,
                xlsCurSheet.Cells(i,17).Value,
                xlsCurSheet.Cells(i,18).Value,
                str(xlsCurSheet.Cells(i,19).Value)
            ]
            xlslist.append(value)
            '''測試用log
            print('ture_value=',value)
            print('ture_xlslist=',xlslist)
            '''
        '''測試用log
        else:
            print('false=',xlsCurSheet.Cells(i,1).Value,type(xlsCurSheet.Cells(i,1).Value))
        '''
        #在畫面中印出第一欄位所有的值。
        #xlsCurSheet.UsedRange.Rows.Count :Excel檔案中的總行數。

    df = pd.DataFrame(xlslist)

    df.columns = ['column1',
        'column1',          
        'column1',
        'column1',   
        'column1',
        'column1',   
        'column1',   
        'column1',   
        'column1',   
        'column1',   
        'column1',   
        'column1'                     
        ]


    xlsBook.Close(SaveChanges=0)
    xlsApp.Quit()
    del xlsApp
    
    '''測試用log
    print('迴圈結束\n\n def內xlslist=',xlslist)
    print('value=',value)
    print('內df=',df)
    '''
    #df
    return xlslist

In [9]:
#擷取信件>>篩選內容

#outlook 擷取特定title

import win32com
import win32com.client
import string
import os

def findFolder(folderName,searchIn):#資料夾搜尋
    try:
        lowerAccount = searchIn.Folders
        for x in lowerAccount:
            if x.Name == folderName:
                print('found it %s'%x.Name)
                objective = x
                return objective
        return None
    except Exception as error:
        print("Looks like we had an issue accessing the searchIn object")
        print(error)
        return None

def CustCDinMail():#處理信件內容資料

    get_path = os.getcwd()
    outlook = win32com.client.Dispatch("Outlook.Application.15").GetNameSpace("MAPI")
    #Outlook.Application.15:連接outlook2013
    
    Folder1 = findFolder('Outlook2020',outlook) #在本機outlook
    Folder2 = findFolder('收件匣',Folder1)
    Folder3 = findFolder('test',Folder2)

    '''測試段落
    #print(outlook)
    print(Folder1)                     
    print(Folder2)                     
    print(Folder3,'\n')
    '''

    #messages = inbox.Items
    messages = Folder3.Items
    message2 = messages.GetLast()
    subject = message2.Subject
    body = message2.body
    sender = message2.Sender
    attachments = message2.Attachments

    '''測試段落
    print('-----擷取內容確認-----')
    #print('1.資料夾:',inbox)
    print('2.未確認',messages)
    print('3.最新',message2)
    print('4.主旨:',subject)
    print('5.內文:\n\n',body)
    print('6.附件:\n',attachments)
    print('-----內容確認結束-----\n\n\n')
    print('執行對應迴圈\n')
    '''

    for m in messages:
        if m.Subject == "title of the mail":
            
            sub=subject
            body=body 
            print('開始切割')
            #print('第1次=',type(CD),len(CD),CD,'\n\n')
            CD=body.split('。')
            #y=0
            #for x in CD:
            #    y=y+1
            #    print('第',y,'個:',x,'\n')
                
            #print('一定是3單一個',CD[2])
                
            
            CD=CD[2].split('\r')
            #print('第2次=',type(CD),len(CD),CD,'\n\n')
            #y=0
            #for x in CD:
            #   y=y+1
            #    print('第',y,'個:',x,'\n')
            
            #print('CD[3]=',CD[3])
            #print('CD[4]=',CD[4])
            #print('一定是4開始到最後',CD[3],CD[4],'\n\n最底端\n\n')
            CustCD=[]
            for x in range(3,len(CD),1):
                #print('len(CD)=',len(CD))
                #print('CD[x]=',CD[x])
                CDf=CD[x].split('   ')
                #print('CDf=',CDf)
                #print('最後切完應該是CustCD=',CDf[1])
                CustCD.append(CDf[1])
            CustCD=set(CustCD) #去除重複
        return CustCD

#print(CustCDinMail())

In [1]:
# -*- coding: UTF-8 -*-
import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')
import win32com.client
import xlwings


def get_excel_date(filename):
    '''
    获得excel里的所有内容，返回list
    :param filename:  excel路径
    :return: list[list[]]
    '''
    app = xlwings.App(visible=False, add_book=True)
    app.display_alerts = False
    app.screen_updating = False
    wb = app.books.open(filename)
    sht = wb.sheets[0]
    ecl = sht.range('A1')
    
    my_date_handler = lambda year, month, day, hour, minute, second, **kwargs: "%04i-%02i-%02i %02i:%02i:%02i" % (
    year, month, day, hour, minute, second)
    fixdate = ecl.current_region.options(index=False, numbers=int, empty='N/A', dates=my_date_handler)
    result = fixdate.value
    wb.close()
    app.quit()
    return result

if __name__ == '__main__':    
    olook = win32com.client.Dispatch("Outlook.Application")
    mail = olook.CreateItem(0)
    mail.Recipients.Add("mail@mail.com") #收件者01
    mail.Recipients.Add("mail@mail.com") #收件者02
    mail.Recipients.Add("mail@mail.com") #收件者03
    mail.Subject = "通知" #標題
    body_html = ""
    body_html = body_html + '<body>Hi all:<br/>下列資料請確認<br/><br/>以上訊息請盡速確認，<br/><br/>謝謝'
    body_html = body_html + '<table width="1" border="1" cellspacing="1" cellpadding="1" height="100">'
    '''取得excel的表格，僅取有資料的部分
    雖然註解掉了但是還是很好用，那天修改以後應該可以用在別的地方
    ecl_list = get_excel_date("C:\\py\\ex1.xlsx")
    '''
    xlslist=[]
    for x in CustCDinMail():
        print(x)
        print(findCustCD(x))
        xlslist.insert(0,findCustCD(x)[0])
        print(xlslist)
    title =['column',
        'column',
        'column',
        'column',
        'column',
        'column',
        'column',
        'column',
        'column',
        'column',
        'column',
        'column',          
        ]
    xlslist.insert(0,title)
    print(xlslist)
    # 表格標題
    for tr_list in xlslist[:1]:
        body_html = body_html + "<tr>"
        for td_list in tr_list:

            body_html = body_html + '<th bgcolor="#AAAAAA" nowrap="nowrap">' + td_list + '</th>'
        body_html = body_html + "</tr>"
    # 表格內容
    for tr_list in xlslist[1:]:
        body_html = body_html + "<tr>"
        for td_list in tr_list:
            body_html = body_html + "<td>" + td_list + "</td>"
        body_html = body_html + "</tr>"
    body_html = body_html + '</table>'
    body_html = body_html + "</body>"
    mail.HTMLBody = body_html
    #mail.Display(true) #跳出信件視窗
    mail.Send()

NameError: name 'CustCDinMail' is not defined